<a href="https://colab.research.google.com/github/Dhaneshkp/DesktopAssistant/blob/main/DATA_processing_expConv2Dv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from math import exp
import numpy as np

# Set a random seed for reproducibility
np.random.seed(42)

# Parameters
n_samples = 288000
unique_risk_classes = ['A', 'B', 'C', 'D']
n_anomalies = 4000
# Generate random ages, durations, and premiums
ages = np.random.randint(65, 70, size=(n_samples, 1))
durations = np.random.randint(20, 25, size=(n_samples, 1))
premiums = ages * np.exp(durations/20) + np.random.randint(10, 50, size=(n_samples, 1))

# Define risk class multipliers
riskclass_multipliers = {
    'A': 1.05,
    'B': 1.1,
    'C': 1.3,
    'D': 1.0
}

# Initialize the result array
result_array = np.empty((n_samples, 1, 3, len(unique_risk_classes)))

# Iterate over samples and risk classes
for i in range(n_samples):
    for j, risk_class in enumerate(unique_risk_classes):
        # Apply risk class multiplier to premium
        premium_with_risk = premiums[i] * riskclass_multipliers[risk_class]

        # Assign age, duration, and premium to the result array
        result_array[i, 0, :, j] = np.concatenate((ages[i], durations[i], premium_with_risk))

anomaly_indices = np.random.choice(n_samples * len(unique_risk_classes), n_anomalies, replace=False)
for idx in anomaly_indices:
    sample_idx = idx // len(unique_risk_classes)
    risk_class_idx = idx % len(unique_risk_classes)
    result_array[sample_idx, 0, 2, risk_class_idx] += np.random.randint(700, 1000)  # Add a large random value to the premium

print(result_array.shape)  # Output: (10000, 1, 3, 4)
result_array=result_array.reshape(-1,24,3,4)
result_array
#anomaly_indices
result_array.shape

(288000, 1, 3, 4)


(12000, 24, 3, 4)

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Conv2DTranspose,Dropout,AveragePooling2D,UpSampling2D
from tensorflow.keras.regularizers import l2
X_train, X_test, train_indices, test_indices = train_test_split(
    result_array,
    np.arange(result_array.shape[0]),  # Create indices for the array
    test_size=0.2,
    random_state=42
)

In [4]:
# prompt: reshape into a darat drame

import numpy as np
import pandas as pd

# ... (Preceding code remains the same)

# Reshape the array into a 2D format
reshaped_array = (X_test).reshape(-1,3*4)

# Create a DataFrame
df = pd.DataFrame(reshaped_array)
# Display the DataFrame
df


,0,1,2,3,4,5,6,7,8,9,10,11
0,66.0,66.0,66.0,66.0,20.0,20.0,20.0,20.0,210.426931,220.447261,260.528581,200.406601
1,65.0,65.0,65.0,65.0,24.0,24.0,24.0,24.0,270.697980,283.588360,335.149880,257.807600
2,66.0,66.0,66.0,66.0,24.0,24.0,24.0,24.0,273.134103,286.140489,338.166032,260.127717
3,66.0,66.0,66.0,66.0,22.0,22.0,22.0,22.0,219.738705,230.202453,272.057445,209.274958
4,69.0,69.0,69.0,69.0,22.0,22.0,22.0,22.0,267.001828,279.716201,330.573692,254.287456
...,...,...,...,...,...,...,...,...,...,...,...,...
57595,68.0,68.0,68.0,68.0,21.0,21.0,21.0,21.0,229.236290,240.152304,283.816359,218.320276
57596,69.0,69.0,69.0,69.0,24.0,24.0,24.0,24.0,273.092471,286.096874,338.114488,260.088068
57597,68.0,68.0,68.0,68.0,23.0,23.0,23.0,23.0,262.244974,274.732830,1174.684253,249.757118
57598,65.0,65.0,65.0,65.0,20.0,20.0,20.0,20.0,208.622735,218.557151,258.294815,198.688319


In [ ]:

#train_indices, test_indices = train_test_split(data_reshaped.index, test_size=0.2, random_state=42)
# Define the CNN architecture
input_shape = (X_train.shape[1], X_train.shape[2],X_train.shape[3])    # Assuming X_train is a 3D array
input_layer = Input(shape=input_shape)
# Define the CNN architecture
input_shape = (X_train.shape[1], X_train.shape[2],X_train.shape[3])    # Assuming X_train is a 3D array
input_layer = Input(shape=input_shape)

conv_layer = Conv2D(4, (2,3), activation="relu",padding='same',kernel_regularizer=l2(0.01))(input_layer)
conv_layer = AveragePooling2D(pool_size=(2,1))(conv_layer)
dropout_layer = Dropout(0.2)(conv_layer)
output_shape = (conv_layer.shape[1], conv_layer.shape[2], conv_layer.shape[3])  # Adjust as needed

# Reshape the output of the convolutional layers
#reshape_layer = Reshape(output_shape)(conv_layer)

# Use transposed convolution (Conv2DTranspose) to upsample and reconstruct
#deconv_layer = Conv2DTranspose(4, (2, 3), activation='relu', padding='same')(reshape_layer)
upsample_layer = UpSampling2D(size=(2, 1))(dropout_layer) # Use UpSampling2D to restore spatial dimensions
deconv_layer = Conv2DTranspose(4, (2, 3), activation='relu', padding='same')(upsample_layer) # Apply transposed convolution
output_layer = Conv2D(X_train.shape[3], (1, 1), activation='linear', padding='same')(deconv_layer)  # Output 4 channels

cnn_model = Model(inputs=input_layer, outputs=output_layer)
cnn_model.compile(optimizer='adam', loss='mse')

# ... (rest of your code for training and prediction)

cnn_model = Model(inputs=input_layer, outputs=output_layer)
cnn_model.compile(optimizer='adam', loss='mse')

# Train the model
history = cnn_model.fit(X_train, X_train,
                        epochs=50,
                        batch_size=32,
                        validation_split=0.1,
                        verbose=1)

# Predict the training data
pred_train = cnn_model.predict(X_train)
mse_train = np.mean(np.power(X_train - pred_train,2), axis=1)



Epoch 1/50
270/270 [==============================] - 2s 5ms/step - loss: 25480.0879 - val_loss: 25352.1738
Epoch 2/50
270/270 [==============================] - 1s 4ms/step - loss: 25370.6465 - val_loss: 25289.6465
Epoch 3/50
270/270 [==============================] - 1s 4ms/step - loss: 25307.8516 - val_loss: 25227.3691
Epoch 4/50
270/270 [==============================] - 1s 4ms/step - loss: 25245.3555 - val_loss: 25165.2695
Epoch 5/50
270/270 [==============================] - 1s 4ms/step - loss: 25183.1680 - val_loss: 25103.3691
Epoch 6/50
270/270 [==============================] - 1s 4ms/step - loss: 25121.1816 - val_loss: 25041.6465
Epoch 7/50
269/270 [============================>.] - ETA: 0s - loss: 25057.1211

In [8]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Reshape

model = Sequential()
model.add(Conv2D(32, (1, 3), activation='relu', input_shape=X_train.shape[1:]))  # Adjusted kernel size
model.add(Flatten())
model.add(Dense(64, activation='relu'))  # Bottleneck layer
model.add(Dense(np.prod(X_train.shape[1:]), activation='linear')) # Output layer for reconstruction
model.add(Reshape(X_train.shape[1:])) # Reshape to original input shape

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, X_train, epochs=10)  # Train on normal data

# Anomaly Detection
pred_test = model.predict(X_test)
mse = np.mean(np.square(X_test - pred_test), axis=(1, 2, 3))

Epoch 1/10
150/150 [==============================] - 1s 3ms/step - loss: 2986.3147
Epoch 2/10
150/150 [==============================] - 0s 3ms/step - loss: 917.6511
Epoch 3/10
150/150 [==============================] - 0s 3ms/step - loss: 877.4870
Epoch 4/10
150/150 [==============================] - 0s 3ms/step - loss: 848.5940
Epoch 5/10
150/150 [==============================] - 0s 3ms/step - loss: 820.9399
Epoch 6/10
150/150 [==============================] - 0s 3ms/step - loss: 791.1042
Epoch 7/10
150/150 [==============================] - 0s 3ms/step - loss: 760.8883
Epoch 8/10
150/150 [==============================] - 0s 3ms/step - loss: 731.5533
Epoch 9/10
150/150 [==============================] - 0s 3ms/step - loss: 703.7039
Epoch 10/10
38/38 [==============================] - 0s 2ms/step


In [ ]:
threshold = np.percentile(mse_train, 99)

# Identify anomalies in the test set
anomalies = mse > threshold

# Print the results
print(f"Threshold: {threshold}")
print(f"Number of anomalies in the test set: {np.sum(anomalies)}")

print(X_test)
print(pred_test)
print(threshold)

Threshold: 818.2270629664945
Number of anomalies in the test set: 0
[[[[ 64.          64.          64.          64.        ]
   [ 24.          24.          24.          24.        ]
   [248.31185721 260.13623136 307.43372797 236.48748306]]

  [[ 65.          65.          65.          65.        ]
   [ 23.          23.          23.          23.        ]
   [240.74666609 252.21079304 298.06730087 229.28253913]]

  [[ 69.          69.          69.          69.        ]
   [ 23.          23.          23.          23.        ]
   [245.08607631 256.75684185 303.439904   233.41531077]]

  ...

  [[ 62.          62.          62.          62.        ]
   [ 20.          20.          20.          20.        ]
   [197.43514703 206.8368207  244.44351537 188.03347336]]

  [[ 68.          68.          68.          68.        ]
   [ 22.          22.          22.          22.        ]
   [237.59745411 248.91161859 294.16827652 226.28328963]]

  [[ 61.          61.          61.          61.        ]
   

In [ ]:
cnn_model.summary()

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 24, 3, 4)]        0         
                                                                 
 conv2d_17 (Conv2D)          (None, 24, 3, 4)          100       
                                                                 
 average_pooling2d_9 (Avera  (None, 12, 3, 4)          0         
 gePooling2D)                                                    
                                                                 
 dropout_7 (Dropout)         (None, 12, 3, 4)          0         
                                                                 
 up_sampling2d_3 (UpSamplin  (None, 24, 3, 4)          0         
 g2D)                                                            
                                                                 
 conv2d_transpose_7 (Conv2D  (None, 24, 3, 4)          100

In [ ]:

# Predict the test data
pred_test = cnn_model.predict(X_test)
mse_test = np.mean(np.power(X_test - pred_test  , 2), axis=1)

# Define a threshold for anomaly detection
threshold = np.percentile(mse_train, 99)

# Identify anomalies in the test set
anomalies = mse_test > threshold

# Print the results
print(f"Threshold: {threshold}")
print(f"Number of anomalies in the test set: {np.sum(anomalies)}")

print(X_test)
print(pred_test)
print(threshold)

38/38 [==============================] - 0s 3ms/step
Threshold: 818.2270629664945
Number of anomalies in the test set: 176
[[[[ 64.          64.          64.          64.        ]
   [ 24.          24.          24.          24.        ]
   [248.31185721 260.13623136 307.43372797 236.48748306]]

  [[ 65.          65.          65.          65.        ]
   [ 23.          23.          23.          23.        ]
   [240.74666609 252.21079304 298.06730087 229.28253913]]

  [[ 69.          69.          69.          69.        ]
   [ 23.          23.          23.          23.        ]
   [245.08607631 256.75684185 303.439904   233.41531077]]

  ...

  [[ 62.          62.          62.          62.        ]
   [ 20.          20.          20.          20.        ]
   [197.43514703 206.8368207  244.44351537 188.03347336]]

  [[ 68.          68.          68.          68.        ]
   [ 22.          22.          22.          22.        ]
   [237.59745411 248.91161859 294.16827652 226.28328963]]

  [[ 

In [ ]:
# Find indices of anomalies in X_test
anomaly_indices = np.where(anomalies)[0]

# Extract the anomalous records from X_test
anomalous_records = X_test[anomaly_indices]

print("Anomalous Records:")
print(anomalous_records)
anomaly_indices

Anomalous Records:
[[[[ 66.          66.          66.          66.        ]
   [ 22.          22.          22.          22.        ]
   [227.08870546 333.50245334 281.15744485 216.27495758]]

  [[ 65.          65.          65.          65.        ]
   [ 23.          23.          23.          23.        ]
   [232.34666609 243.41079304 287.66730087 221.28253913]]

  [[ 67.          67.          67.          67.        ]
   [ 24.          24.          24.          24.        ]
   [257.72022551 269.99261721 319.08218397 245.44783382]]

  ...

  [[ 67.          67.          67.          67.        ]
   [ 22.          22.          22.          22.        ]
   [221.31807978 231.85703596 274.01286069 210.7791236 ]]

  [[ 67.          67.          67.          67.        ]
   [ 24.          24.          24.          24.        ]
   [256.67022551 268.89261721 317.78218397 244.44783382]]

  [[ 68.          68.          68.          68.        ]
   [ 20.          20.          20.          20.     

array([   4,    4,   10,   10,   11,   23,   23,   25,   25,   47,   54,
         66,   90,   93,   96,   97,  111,  115,  123,  126,  131,  149,
        150,  160,  174,  180,  188,  209,  211,  213,  214,  215,  215,
        217,  224,  226,  231,  270,  274,  292,  300,  306,  326,  327,
        336,  344,  345,  346,  360,  361,  372,  373,  380,  381,  397,
        410,  414,  419,  420,  432,  440,  442,  444,  446,  446,  454,
        456,  469,  470,  474,  476,  477,  480,  485,  486,  492,  499,
        503,  508,  511,  526,  526,  538,  579,  582,  584,  585,  596,
        602,  611,  627,  630,  656,  662,  668,  672,  675,  691,  698,
        699,  708,  710,  722,  726,  732,  741,  769,  774,  774,  777,
        780,  783,  785,  785,  787,  795,  797,  801,  803,  803,  804,
        808,  809,  810,  811,  812,  816,  816,  825,  828,  828,  829,
        829,  833,  846,  853,  865,  868,  879,  887,  895,  903,  905,
        937,  938,  952,  953,  968,  990,  991,  9

In [ ]:
# Create a copy of the original test data from the NumPy array
test_indices
anomalies


array([[[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]],

       [[False, False, False, False],
        [False, False, False, False],
        [False,  True, False, False]],

       [[False, False, False, False],
        [False, False, False, False],
        [False,  True, False, False]],

       ...,

       [[False, False, False, False],
        [False, False, False, False],
        [False,  True, False, False]],

       [[False, False, False, False],
        [False, False, False, False],
        [False,  True, False, False]],

       [[False, False, False, False],
        [False, False, False, False],
        [ True,  True, False, False]]])

In [ ]:
# prompt: crate an isolation forest to detect anomalies in teh result_df_encoded and fina the index of anomalies in teh data and crate a data frame like test_df_with_anomalies , use the xtrain and xtest etc using the xtrain and xtest indices created in the previuos code, and for isolation forest i think we need y test and train so creat theam , if we drop premium rate then thas sdropiing the ioput

import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

# Assuming 'result_df_encoded' is your DataFrame from the preceding code

# Prepare the data for Isolation Forest
X = result_df_encoded.drop('Premium_rate', axis=1).values  # Assuming 'Premium_rate' is the target variable

# Split the data into training and test sets (using the same indices as before)
X_train = X[train_indices]
X_test = X[test_indices]
y_train = result_df_encoded.loc[train_indices, 'Premium_rate'].values
y_test = result_df_encoded.loc[test_indices, 'Premium_rate'].values

# Create an Isolation Forest model
model = IsolationForest(contamination=0.05, random_state=42)  # Adjust contamination as needed

# Fit the model to the training data
model.fit(X_train)

# Predict anomalies on the test data
y_pred = model.predict(X_test)

# Identify anomalies (-1 indicates an anomaly)
anomalies = y_pred == -1

# Create a copy of the original test DataFrame
test_df_with_anomalies_if = result_df_encoded.iloc[test_indices].copy()

# Add an 'anomaly' column to the test DataFrame
test_df_with_anomalies_if['anomaly'] = anomalies

# Print the results
print(f"Number of anomalies in the test set (Isolation Forest): {np.sum(anomalies)}")

# Display the test DataFrame with anomaly tags
test_df_with_anomalies_if


Number of anomalies in the test set (Isolation Forest): 455


,Age,Duration,Premium_rate,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2,anomaly
21185,22.0,4.0,8.359788,0.0,0.0,1.0,0.0,False
21081,41.0,2.0,4.408889,1.0,0.0,0.0,0.0,False
38072,35.0,9.0,9.813333,0.0,0.0,0.0,1.0,False
37127,21.0,9.0,7.733333,0.0,0.0,0.0,1.0,False
22786,38.0,3.0,5.911330,0.0,0.0,0.0,1.0,False
...,...,...,...,...,...,...,...,...
25543,42.0,3.0,8.059259,0.0,0.0,1.0,0.0,False
23315,28.0,5.0,7.714286,0.0,0.0,1.0,0.0,False
8920,41.0,7.0,12.370370,0.0,0.0,1.0,0.0,False
21822,47.0,5.0,15.828571,0.0,0.0,1.0,0.0,False
